In [2]:
from pydantic import BaseModel

In [14]:
from io import BytesIO
from typing import Any, Literal
from pydantic import SerializationInfo, model_serializer, BaseModel



class Schas(BaseModel):
    name: str
    age: int

    class Config:
        arbitrary_types_allowed=True
        
    @model_serializer(mode="plain")
    def ser_model(self, info: SerializationInfo) -> str|bytes|BytesIO:

        return BytesIO(b"asdasdasda")

    # def model_dump(
    #     self,
    #     *,
    #     mode: Literal['json', 'python'] | str = 'python',
    #     include: Any = None,
    #     exclude: Any = None,
    #     by_alias: bool = False,
    #     exclude_unset: bool = False,
    #     exclude_defaults: bool = False,
    #     exclude_none: bool = False,
    #     round_trip: bool = False,
    #     warnings: bool = True,
    # ) -> str: ...

s = Schas(name='John', age=25)

print(s.model_dump())

SerializationIterator(index=0, iterator=<_io.BytesIO object at 0x72e274110950>)


In [13]:
s.model_dump()

SerializationIterator(index=0, iterator=<_io.BytesIO object at 0x72e27430e200>)

In [28]:
ss=s.model_dump(mode="bytesio", serialize_as_any=True)
ss

/home/phil/dev/projects/edutap/edutap.wallet_apple/venv/lib/python3.12/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `str` but got `BytesIO` with value `<_io.BytesIO object at 0x7bb2c43c2610>` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `bytes` but got `BytesIO` with value `<_io.BytesIO object at 0x7bb2c43c2610>` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


SerializationIterator(index=0, iterator=<_io.BytesIO object at 0x7bb2c43c2610>)

In [29]:
next(ss)

/tmp/ipykernel_7587/3438019279.py:1: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `str` but got `BytesIO` with value `<_io.BytesIO object at 0x7bb2c43c2610>` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `bytes` but got `BytesIO` with value `<_io.BytesIO object at 0x7bb2c43c2610>` - serialized value may not be as expected
  next(ss)


b"serialized string info: SerializationInfo(include=None, exclude=None, context=None, mode='bytesio', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=True)"

In [29]:

from collections.abc import Sequence
from typing import overload


@overload
def double(input_: int) -> int: ...


@overload
def double(input_: Sequence[int]) -> list[int]: ...


def double(input_: int | Sequence[int]) -> int | list[int]:
    if isinstance(input_, Sequence):
        return [i * 2 for i in input_]
    return input_ * 2

In [41]:
from pydantic import field_validator, validator
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_prefix="TEST_",
        case_sensitive=False,
        # env_file=".env",
        # env_file_encoding="utf-8",
        extra="allow",
    )    

    val: str
    ii: int

    @field_validator("vals", mode="before")
    @classmethod
    def check_vals(cls, v:str) -> list[str]:
        print(f"validate: {v}")
        return v.split(",")
    
    vals: list[str]



In [42]:
import os
os.environ["TEST_VAL"] = "test" 
os.environ["TEST_II"] = "1"
os.environ["TEST_VALS"] = "a,b,c"


In [43]:
Settings()

SettingsError: error parsing value for field "vals" from source "EnvSettingsSource"

In [44]:
from pydantic import BaseModel, field_validator
class Schas(BaseModel):
    name:str

    @field_validator("hobbies", mode="before")
    def check_hobbies(cls, v):
        if not isinstance(v, list):
            return v.split(",") 
        return v

    hobbies:list[str]

s=Schas(name="John", hobbies="a,b,c")

In [37]:
s

Schas(name='John', hobbies=['a', 'b', 'c'])

In [45]:
from pydantic import BaseModel, Field, SchemaValidator

ImportError: cannot import name 'SchemaValidator' from 'pydantic' (/home/phil/dev/projects/edutap/edutap.wallet_apple/venv/lib/python3.12/site-packages/pydantic/__init__.py)

In [47]:
from pydantic_core import SchemaValidator, SchemaSerializer

In [50]:
from pydantic import BaseModel, Field, model_serializer
from typing import Literal
from io import BytesIO
import json

class ExampleModel(BaseModel):
    name: str
    age: int

    @model_serializer(mode='bytesio')
    def to_bytesio(self, mode: Literal['bytesio'], **kwargs) -> BytesIO:
        # Serialisiere das Modell in JSON
        json_data = self.json().encode('utf-8')
        # Erstelle ein BytesIO-Objekt aus den JSON-Daten
        return BytesIO(json_data)

# Beispielverwendung
example = ExampleModel(name="John Doe", age=30)
bytesio_obj = example.to_bytesio(mode='bytesio')
print(bytesio_obj.getvalue())  # Ausgabe: b'{"name": "John Doe", "age": 30}'

AssertionError: invalid mode: 'bytesio', expected 'plain' or 'wrap'

In [65]:
from pydantic import BaseModel, Field, SerializerFunctionWrapHandler, model_serializer, SerializationInfo
from typing import Literal, Any, Dict


class ExampleModel(BaseModel):
    name: str
    age: int

    @model_serializer(mode='wrap')
    def to_wrapped(self: Any, h: SerializerFunctionWrapHandler, info: SerializationInfo) -> Dict[str, Any]:
        # Serialisiere das Modell in ein Wörterbuch
        # data = self.dict()
        # Füge zusätzliche Informationen hinzu
        wrapped_data = {
            "type": "example_model",
            "attributes": dict(a=1,b=2)
        }
        return wrapped_data

# Beispielverwendung
example = ExampleModel(name="John Doe", age=30)
# wrapped_data = example.to_wrapped(mode='wrap')
# print(wrapped_data)

example.model_dump(mode="zip")

{'type': 'example_model', 'attributes': {'a': 1, 'b': 2}}

In [2]:
from io import BytesIO
from pydantic import BaseModel, Field, SerializerFunctionWrapHandler, model_serializer, SerializationInfo
from typing import Literal, Any, Dict


class ExampleModel(BaseModel):
    name: str
    age: int
    
    class Config:
        arbitrary_types_allowed = True
        # necessary for the model_serializer can have return type other than str|dict
        # TODO: check if this is correct

    @model_serializer(mode='wrap')
    def to_wrapped(self: Any, h: SerializerFunctionWrapHandler, info: SerializationInfo) -> BytesIO:
        # Serialisiere das Modell in ein Wörterbuch
        # data = self.dict()
        # Füge zusätzliche Informationen hinzu
        wrapped_data = {
            "type": "example_model",
            "attributes": dict(a=1,b=2)
        }

        return BytesIO(b"asdasdasasdasd")
        return wrapped_data

# Beispielverwendung
example = ExampleModel(name="John Doe", age=30)
# wrapped_data = example.to_wrapped(mode='wrap')
# print(wrapped_data)

r=example.model_dump(mode="zip")

In [4]:
r.read()

b'asdasdasasdasd'

In [70]:
[x for x in r]

[b'asdasdasasdasd']

In [73]:
r.read

AttributeError: 'pydantic_core._pydantic_core.SerializationIterator' object has no attribute 'read'

In [1]:
generator

NameError: name 'generator' is not defined

In [2]:
from typing import Any, Dict, Literal, Generator

In [2]:
from io import BytesIO
import types

bb=BytesIO(b"asdadad")
isinstance(bb, types.GeneratorType)

False

In [3]:
isinstance(BytesIO(b"adasd"), types.GeneratorType)

False

In [4]:
from collections.abc import Iterable
isinstance(BytesIO(b"adasd"), Iterable)

True

In [6]:
bb.__class__.__bases__

(_io._BufferedIOBase,)

In [7]:
f = open("/tmp/tmp-ccc.xpi")

In [11]:
f.__class__.__bases__[0].__bases__

(_io._IOBase,)

In [12]:
from io import IOBase

In [13]:
isinstance(f, IOBase)

True